# **股票篩選系統 For建州老師**  
篩選系統目前版本 : Ver1.6  
篩選系統更新日期 : 2022.06.29  
篩選系統更新內容 :   
-更動股票清單索引取值方式


#**■股票篩選系統相關紀錄■**  
[Google表單連結](https://docs.google.com/spreadsheets/d/1qOvx4zhEKHYXc_NPdXR3tyCowb8Jd8xnYIEu7a-8gIc/edit?usp=sharing)  
[系統測試期間篩選紀錄(符合精選條件)](https://rdsqys.com/stock.html)

# **■股票篩選系統使用說明及注意事項■**
## **使用說明**
1.   將鼠標滑到 [#匯入函式資料庫] 旁邊的中括號位置
2.   出現三角形執行圖示後請點一下，以執行篩選系統
3.   執行結果請詳見[圖文解說](https://bacons.cc/ms/0757.jpg)

## **注意事項**
*   本系統尚有許多不盡之處，因此我將會不定時的更新系統，老師每次所看到的系統可能會有些許不同。
*   證交所是有流量限制的，所以系統的預設是每次執行會待機15秒(約為4次篩選/分鐘)，因此跑完整個系統將可能花費您約10小時的執行時間。目前我的解決辦法是多系統並行篩選，但由於多系統篩選意味著需要多個Google帳號的權限，再加上要顧及多系統的運行情況，對您可能較不方便，因此我會在上方的Google表單連結中放上我當天多系統執行的結果，以供老師快速做資料分析。
*   更多關於本系統的說明，還勞煩老師前往[GitHub專案頁面](https://github.com/bacon166539/stock)查看

#**■Line Notify相關說明■**  
*   若老師想從Line上遠程查看最新篩選結果，請點旁邊[加入群組](https://line.me/ti/g/tgJye_s-2S)，裡面我已經先將機器人設置完成，無須額外訊息串接的設定。
*   Line Notify的預設通知條件是，個股符合精選條件或當沖精選條件時才會發出通知。
*   因當日符合通知條件的個股可能很多，為避免打擾您，建議您將群組關閉提醒。
*   若我當日有使用多系統並行篩選，篩選結果也會通知給您。




# **■買賣出說明■**
### **(因文字可能不好理解，建議您可以看[圖文版](https://bacons.cc/d/3112)。)**
第一次賣出日為(n+2天正常開市日)日，若(n+2天正常開市日)日個股股價大於(n)日之收盤價，則賣出。反之，將續抱該個股至第二賣出日。

第二次賣出日為以一期間之最後交易日(m)，往下週找最後開市日(o)，若攤還日期間(m ~ o)，有大於(n)日之開盤價之日(p)者，則(p)日賣出。反之，將續抱該個股至第三賣出日。

第三次賣出日為以上週最後開市日(o)，往下週找最後開市日(q)，若攤還日期間(o ~ q)，有大於(n)日之開盤價之日(r)者，則(r)日賣出。反之，全數於(n)日購入之股票於(q)日停損。

# **■系統篩選法則■**
### **下述條件暫不考慮是否得以實際購入，例如:隔日開盤漲/跌停或因特殊原因暫緩交易等…。**
基礎條件(符合條件清單):現價(收盤價)必須大於28日均價  
精選條件(符合精選條件清單):符合前述條件，且當日(近一日)成交量需大於過去五日最高成交量  
當沖精選條件(符合當沖精選條件清單):符合前述條件，且個股是為可以當沖之商品  





# **■股票篩選系統程式主體■**

In [ ]:
#匯入函式資料庫
!pip install twstock
!pip install yfinance
import twstock as t 
import pandas as p 
from bs4 import BeautifulSoup 
import requests 
import time as ti
import os as o



def web(url):#定義網頁查詢函式
    source = requests.get(url, headers={'Connection':'close'}) #連線到指定的網站
    soup = BeautifulSoup(source.content, 'lxml', from_encoding='utf-8') #讀取這個網頁的內容
    return soup #最終結果:回傳網頁內容

def post_data(message, token):#Line notify傳訊息
    try:
        url = "https://notify-api.line.me/api/notify"#api網址
        headers = {
            'Authorization': f'Bearer {token}'
        }
        payload = {
            'message': message
        }
        response = requests.request(
            "POST",
            url,
            headers=headers,
            data=payload
        )
        if response.status_code == 200:
            print(f"Success -> {response.text}")
    except Exception as _:
        print(_)#狀態

#####取得上市股#####
data = p.read_html('http://isin.twse.com.tw/isin/C_public.jsp?strMode=2', encoding='big5hkscs', header=0)[0]
data = p.DataFrame(data["有價證券代號及名稱"])
data = data["有價證券代號及名稱"].astype(str)

#####取得上櫃股#####
data2 = p.read_html('http://isin.twse.com.tw/isin/C_public.jsp?strMode=4', encoding='big5hkscs', header=0)[0]
data2 = p.DataFrame(data2["有價證券代號及名稱"])
data2 = data2["有價證券代號及名稱"].astype(str)

#####取得可當沖股#####
TW50 = [1101, 1216, 1301, 1303, 1326, 1590, 2002, 2207, 2227, 2303, 2308, 2317, 2327, 2330, 2352, 2357, 2379, 2382, 2395, 2408, 2409, 2412, 2454, 2603, 2609, 2610, 2615, 2801, 2880, 2881, 2882, 2883, 2884, 2885, 2886, 2887, 2891, 2892, 2912, 3008, 3034, 3037, 3045, 3711, 4904, 5871, 5876, 5880, 6415, 6505, 8046, 8454, 9910]
RICH50 = [1565, 1785, 3081, 3105, 3152, 3211, 3218, 3227, 3228, 3260, 3264, 3293, 3324, 3374, 3529, 3552, 3680, 3707, 4105, 4123, 4128, 4736, 4966, 5274, 5289, 5347, 5351, 5371, 5425, 5483, 6104, 6121, 6138, 6147, 6182, 6274, 6411, 6488, 6510, 6547, 6548, 6561, 6732, 6741, 8069, 8086, 8255, 8299, 8358, 8436]
MID100A = [1102,1210,1227,1229,1402,1434,1440,1451,1476,1477,1504,1560,1605,1717,1718,1722,1802,1907,2006,2014,2023,2027,2049,2105,2201,2301,2312,2313,2324,2337,2344,2347,2353,2354,2356,2360,2371,2376,2377,2385,2441,2449,2458,2474,2478,2492,2498,2542,2606,2618,2633,2637,2809,2812,2834,2845,2888,2889,2890,2915,3023,3035,3036,]
MID100B = [3044,3051,3189,3231,3406,3443,3481,3532,3533,3653,3661,3702,3714,4137,4919,4938,4958,4961,5269,5522,6005,6116,6176,6213,6239,6271,6409,6531,6592,6669,6719,6770,6781,8464,9904,9914,9917,9921,9941,9945]
Ulti_list = RICH50  + MID100A + MID100B + TW50 

#####創建空串列#####
stocklist_data = []#上市篩選股票清單
stocklist_data2 = []#上櫃篩選股票清單
stocklist = []#篩選股票清單
stock28list = []#符合條件清單
amu28list = []#符合精選條件清單
ulti28list = []#符合當沖精選條件清單

#####代數預設指定#####
eps_check = 0 #eps資料檢查
divi_check = 0 #dividend資料檢查
self_divi_check1 = 0 #dividend資料自檢查1
self_divi_check2 = 0 #dividend資料自檢查2
self_eps_check1 = 0 #eps資料自檢查1
self_eps_check2 = 0 #eps資料自檢查2
characters = "%," #設定特定字符
rps_result = str('查無計算所需資料')#rps_result初始給值
data_check = 0 #上市篩選索引檢查
data2_check = 0 #上櫃篩選索引檢查

#####抓找上市篩選股票清單#####
for x in data.iloc:#個別抓資料
    try:
        if x == "股票":#找到"股票"索引
            data_check = 1 #上市篩選索引開始
        if x == "上市認購(售)權證":#找到"上市認購(售)權證"索引
            data_check = 0 #上市篩選索引結束
        if data_check == 1:#如果上市篩選索引開始
            stockid = x
            stocklist_data.append(stockid)#匯入清單
    except:
        continue
stocklist_data.remove("股票")#清除清單中的"股票"

#####抓找上櫃篩選股票清單#####
for z in data2.iloc:#個別抓資料
    try:
        if z == "股票":#找到"股票"索引
            data2_check = 1 #上櫃篩選索引開始
        if z == "特別股":#找到"特別股"索引
            data2_check = 0 #上櫃篩選索引結束
        if data2_check == 1:#如果上櫃篩選索引開始
            stockid = z
            stocklist_data2.append(stockid)#匯入清單
    except:
        continue
stocklist_data2.remove("股票")#清除清單中的"股票"

#####整合上市上櫃篩選股票清單#####
stocklist = stocklist_data + stocklist_data2#整合

#####Line Notify通知#####
if __name__ == "__main__":
  token = "STiL2z0akXpZt1G63Z59VhmW1sFWlR2MtO3q5pUZLXV" # Token
  message = {"\n⚙系統初始化設定完成\n🤖開始篩選股票\n提醒您，相關操作說明請詳閱記事本內的連結喔!"}     # 要發送的訊息
  post_data(message, token)

print("系統設定完成...\n開始篩選股票\n===============================")
print("實時符合當沖精選條件的股票:")
#####篩選股票迴圈#####
for x in stocklist:#個別抓資料
    try:
        ti.sleep(15)#待機15秒
        stk = x[:4]#取資料前4碼
        stk = str(stk)
        #####查詢最新股價#####
        price_url = 'https://tw.stock.yahoo.com/quote/' + stk #查詢股價的網址
        systime = ti.localtime()#取得查價時間
        span = web(price_url).find_all("span")#抓span資料
        for p in span:
            if p.text == '成交':#抓成交sapn
                price = float(span[span.index(p) + 1].string) #股價sapn
                lastprice = price#給值
            else:
                continue
        stk_price = str('') #股價標籤初始給值
        if lastprice >= 100 and lastprice < 250:
            stk_price = str('🟢')
        elif lastprice >= 250 and lastprice < 500:
            stk_price = str('🔴')
        elif lastprice >= 500:
            stk_price = str('⚫')
        #####查詢最新每股盈餘季增率%#####
        eps_price_url = 'https://tw.stock.yahoo.com/quote/' + stk + '/eps' #查詢每股盈餘的網址
        eps_span = web(eps_price_url).find_all("span") #抓span資料
        for p in eps_span:
            if p.text == '年增率%':#抓年增率%sapn
                eps_result = str(eps_span[eps_span.index(p) + 4].string) #季增率%sapn
                for d in range(len(characters)): #個別抓取特定字符
                    eps_result = eps_result.replace(characters[d],"") #符合字符替換
                eps_result = float(eps_result) #轉換格式
                eps_yield = eps_result #給值
                self_eps_check1 = 1 #自檢1成功給值
            elif p.text == '及': #抓必定有的span
                eps_non = str(span[span.index(p) + 1].string) #隨便抓
                self_eps_check2 = 1 #自檢2成功給值
            else:
                continue
        if self_eps_check2 == 1: #自檢2
            if self_eps_check1 == 1: #自檢1
                eps_check = 1 #eps檢查成功給值
        #####查詢最新每股股息(全年)#####
        divi_price_url = 'https://tw.stock.yahoo.com/quote/' + stk + '/dividend' #查詢股息的網址
        span = web(divi_price_url).find_all("span")#抓span資料
        for p in span:
            if p.text == '股票股利':#抓股票股利sapn
                divi_result1 = float(span[span.index(p) + 4].string) #現金股利sapn
                divi_result2 = float(span[span.index(p) + 5].string) #股票股利sapn
                divi_result = divi_result1 + divi_result2 #計算總股息
                stkdiv = divi_result #給值
                self_divi_check1 = 1 #自檢1成功給值
            elif p.text == '及': #抓必定有的span
                divi_non = str(span[span.index(p) + 1].string) #隨便抓
                self_divi_check2 = 1 #自檢2成功給值
            else:
                continue
        if self_divi_check2 == 1: #自檢2
            if self_divi_check1 == 1: #自檢1
                divi_check = 1 #divi檢查成功給值
        #####取得個股各項資料#####
        stock = t.Stock(stk) #送出查詢至twse並取得回傳資料
        price = stock.price#從回傳資料取價格並給值
        mixresult = str('查無計算所需資料') #mixresult初始給值      
        if divi_check == 1: #divi檢查成功
            if eps_check == 1: #eps檢查成功
                b = t.BestFourPoint(stock)#從回傳資料分析四大買賣點
                mixresult = b.best_four_point() #綜合分析 
                result = mixresult[0]#從分析取得買賣點
        #########################
        #####計算28MA#####
        average = stock.moving_average(price, 28)#取價並依序計算並給值
        last28price = average[3]#從值取第4位價格
        time = ti.strftime("%Y/%m/%d, %H:%M:%S", systime)#重新編時間格式
        #########################
        #####查詢成交量#####
        amount = stock.capacity#從回傳資料取成交量並給值
        lastamu = amount[30]#從值取第31位成交量
        recentamu = amount[26:]#從值取後5日成交量
        maxamu = max(recentamu)#從值取後5日最大成交量
        #########################
        #####是否站上28均線#####
        stk_ulti = str('⛔') #是否可當沖貼圖初始給值
        if lastprice >= last28price:#是，站上28均線
            stock28list.append(x)#匯入資料至符合條件清單
            #####比較成交量#####
            if lastamu >= maxamu:#是，成交量大
                amu28list.append(x)#匯入資料至符合精選條件清單
                stk = int(stk)#轉換stk為整數
                #####是否為可當沖個股#####
                if stk in Ulti_list:#是
                    ulti28list.append(x)#匯入資料至符合當沖精選條件清單
                    stk_ulti = str('🉑')
                if __name__ == "__main__":#是
                    stk = str(stk)#轉換stk為文字
                    print(time)#輸出取價時間
                    print('股票代碼',stk)#輸出股票代碼
                    #####計算市場冷熱程度#####
                    if divi_check == 1: #divi檢查成功
                        if eps_check == 1: #eps檢查成功
                            div_yield = ("{:.2%}".format(stkdiv/lastprice)) #divi檢查成功
                            for e in range(len(characters)):
                                div_yield = div_yield.replace(characters[e],"")
                            div_yield = float(div_yield)
                            final_eps = (eps_yield/div_yield)/100
                            if final_eps >= 0.012:
                                final_eps = final_eps * 100
                                final_eps = ("{:.2f}".format(final_eps))
                                rps_result = ('【市場過冷，考慮建倉】\n【總報酬本益比:'+str(final_eps)+"倍】")#輸出股票代碼
                            elif final_eps <= 0.007 and final_eps >= 0:
                                final_eps = final_eps * 100
                                final_eps = ("{:.2f}".format(final_eps))
                                rps_result = ('【市場過熱，考慮清倉】\n【總報酬本益比:'+str(final_eps)+"倍】")#輸出股票代碼
                            elif final_eps <= 0.011 and final_eps >= 0.008:
                                final_eps = final_eps * 100
                                final_eps = ("{:.2f}".format(final_eps))
                                rps_result = ('【市場普普，暫且觀望】\n【總報酬本益比:'+str(final_eps)+"倍】")#輸出股票代碼
                            else:
                                final_eps = final_eps * 100
                                final_eps = ("{:.2f}".format(final_eps))
                                rps_result = ('【無法估計，因數值無意義】\n【總報酬本益比:'+str(final_eps)+"倍】")
                    print(rps_result)#輸出股票代碼
                    #####Line Notify通知#####
                    if __name__ == "__main__":
                        token = "STiL2z0akXpZt1G63Z59VhmW1sFWlR2MtO3q5pUZLXV" # Token
                        message = {"\n📊股票名稱\n【"+ str(x) +"】"+ str(stk_ulti)+ str(stk_price)+"\n💵最新價格\n【"+ str(lastprice) +" 元】\n♨市場冷熱度\n"+ str(rps_result) +"\n🔍簡易技術分析\n【"+ str(mixresult) +"】\n🕰取價時間\n【"+ str(time)+"】"}     # 要發送的訊息
                        post_data(message, token)
                elif stk not in Ulti_list:
                    continue
        #########################
    except:
        continue
#########################
print("===============================")
print(stock28list)#輸出符合條件清單
print("----------------------")
print(amu28list)#輸出符合精選條件清單
print("----------------------")
print(ulti28list)#輸出符合當沖精選條件清單
print("----------------------")
systime2 = ti.localtime()#取得結束時間
time2 = ti.strftime("%Y/%m/%d, %H:%M:%S", systime2)#重新編時間格式
#####Line Notify通知#####
if __name__ == "__main__":
    token = "STiL2z0akXpZt1G63Z59VhmW1sFWlR2MtO3q5pUZLXV" # Token
    message = {"\n⚙系統已完成股票篩選\n🕰系統時間\n【"+ str(time2)+"】\n符合當沖精選條件之股票清單\n"+str(ulti28list)}     # 要發送的訊息
    post_data(message, token)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 7.5 MB/s 
     |████████████████████████████████| 62 kB 836 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you h